In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Rush,Red Sector A,All that we can do is just survive \r\nAll th...
1,Fatboy Slim,5 Reasons,Everytime I try to explain \r\nall you want t...
2,Chaka Khan,And The Melody Still Lingers On (Night In Tuni...,A long time ago in the 40's \r\nDizzy and Bir...
3,"Harry Connick, Jr.",Don't Get Around Much Anymore,Missed the saturday dance \r\nHeard they crow...
4,Frank Sinatra,Dear Little Boy Of Mine,"(Boy of mine, boy of mine, dear little boy of ..."
5,Enya,I Want Tomorrow,"Dawn breaks, there is blue in the sky. \r\nYo..."
6,John Mellencamp,Like A Rolling Stone,Once upon a time you dressed so fine \r\nThre...
7,Michael Buble,I Wish You Love,I wish you bluebirds in the spring \r\nTo giv...
8,Freddie King,Lowdown In Lodi,"Just about a year ago, I set out on the road ..."
9,Morrissey,Jack The Ripper,"Oh, you look so tired \r\nMouth slack and wid..."


In [9]:
df['text'][0]

"All that we can do is just survive  \r\nAll that we can do to help ourselves is stay alive  \r\n  \r\nRagged lines of ragged grey  \r\nSkeletons, they shuffle away  \r\nShouting guards and smoking guns  \r\nWill cut down the unlucky ones  \r\n  \r\nI clutch the wire fence until my fingers bleed  \r\nA wound that will not heal  \r\nA heart that cannot feel  \r\nHoping that the horror will recede  \r\nHoping that tomorrow we'll all be freed  \r\n  \r\nSickness to insanity  \r\nPrayer to profanity  \r\nDays and weeks and months go by  \r\nDon't feel the hunger  \r\nToo weak to cry  \r\n  \r\nI hear the sound of gunfire at the prison gate  \r\nAre the liberators here?  \r\nDo I hope or do I fear?  \r\nFor my father and my brother, it's too late  \r\nBut I must help my mother stand up straight  \r\n  \r\nAre we the last ones left alive?  \r\nAre we the only human beings to survive?\r\n\r\n"

In [10]:
df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.03674875, 0.        , ..., 0.00340236, 0.03006501,
       0.04298052])

In [22]:
df[df['song'] == 'As Good As New']

,artist,song,text


In [26]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [28]:
import pickle


In [29]:
pickle.dump(similarity,open('similarity.pkl','wb'))


In [41]:
pickle.dump(df,open('df.pkl','wb'))